# Covert Channel Machine Learning

## Loading data and preprocessing

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
import os
# Location of .obj files in Google Drive
data_location = '/content/drive/MyDrive/CC Machine Learning/HTTP Stuff'
os.chdir(data_location)

In [6]:
!ls | grep obj

X_http.obj
X_http_simp.obj
y_http.obj
y_http_simp.obj


Now we are in the proper location to access the .obj files

In [15]:
import pickle
import numpy as np

X_file = 'X_http_simp.obj'
y_file = 'y_http_simp.obj'

standard_length = 12

# Load everything like this because pickling buffer has been used
with open(X_file, 'rb') as f:
  # initilize 2D array
  X = np.empty((0, standard_length), dtype=int)
  captures_loaded = 0
  while True:
    try:
      new_arr = pickle.load(f)
      if new_arr.size > standard_length:
        new_arr = new_arr[:standard_length]
      elif new_arr.size < standard_length:
        # pad to required length
        while new_arr.size < standard_length:
          new_arr = np.append(new_arr, 0)

      # add to our main 2D array
      X = np.vstack([X, new_arr])
      captures_loaded += 1
    except EOFError:
      break

print(f'Captures loaded: {captures_loaded}')
print(f'X shape: {X.shape}')

# Load y normally
with open(y_file, 'rb') as f:
  y = np.empty(0)
  while True:
    try:
      y = np.append(y, pickle.load(f))
    except EOFError:
        break

print(f'y shape: {y.shape}')

Captures loaded: 2000
X shape: (2000, 12)
y shape: (2000,)


* X - 2D numpy array (each capture, each packet in that capture)
* y - 1D numpy array for targets (target)

In [16]:
# Now need to convert y to categorical
from keras.utils import to_categorical
y = to_categorical(y)

In [17]:
# Print shape information
print(X.shape, '\n', y.shape, sep='')

(2000, 12)
(2000, 2)


In [18]:
first_average = 0
second_average = 0
for i, capture in enumerate(X):
  this_avg = np.average(capture)
  if i < 1000:
    first_average += this_avg
  else:
    second_average += this_avg

print('Inactive average entropy:', first_average/1000)
print('Active average entropy:', second_average/1000)

Inactive average entropy: 3.7191481682688434
Active average entropy: 3.6359645905144227


In [19]:
# Generate train and test datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)

In [21]:
# Scale data to fit between 0 and 1 range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(X_train)
scaler.fit(X_test)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Building the network with Keras

In [23]:
# Import Keras elements needed for the model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import GlobalMaxPooling1D
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import InputLayer
from keras.layers import Add
from keras.layers import LayerNormalization
from keras.layers import MultiHeadAttention
from keras.optimizers import Adam
import tensorflow as tf

In [24]:
# Define important dimensions and other settings to be used in building the model
# TODO: maybe need to set n_packets to the max amount of packets in one of the captures and then fill 0s for captures with less packets than that
n_entropies = X.shape[1]                   # max no. of packets in one capture
n_outputs = y_train.shape[1]               # either yes or no CC
verbose = 0
epochs = 10
batch_size = 32

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, LSTM, Dense, Bidirectional, InputLayer

# Define the input shape and number of outputs
sequence_length = n_entropies  # Replace with your sequence length
n_outputs = 2  # Since you have two classes

# Create the model
model = Sequential()

# Input Layer
model.add(InputLayer(input_shape=(sequence_length, 1)))

# First Conv Block
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(filters=64, kernel_size=5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# Second Conv Block
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.4))

# Third Conv Block
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Bidirectional LSTM Layers
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))

# Fully Connected Layers
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(n_outputs, activation='softmax'))

# Compile the Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the Model
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 12, 64)              │             384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 12, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 12, 64)              │          20,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 12, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 6, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 6, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 6, 128)              │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 6, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 6, 128)              │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 6, 128)              │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 3, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 3, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 3, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 3, 256)              │         196,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 3, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,018,690 (3.89 MB)

 Trainable params: 1,016,898 (3.88 MB)

 Non-trainable params: 1,792 (7.00 KB)

In [28]:
# TESTING THE MODEL
import time

repeats = 10
total = 0
total_time = 0
for i in range(repeats):
    print("Test", i+1, "... ", end="")
    start_time = time.time()
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)[1]
    time_elapsed = time.time() - start_time
    print(f"{accuracy*100:.2f}% \t Time: {time_elapsed:.2f}s")
    total += accuracy*100
    total_time += time_elapsed
total_accuracy = total/repeats
print(f"Total: {total_accuracy:.2f}%")
print(f"Average time: {total_time/repeats:.2f}s")


Test 1 ... 75.50% 	 Time: 28.22s
Test 2 ... 77.50% 	 Time: 28.77s
Test 3 ... 76.00% 	 Time: 28.62s
Test 4 ... 78.50% 	 Time: 28.47s
Test 5 ... 72.50% 	 Time: 30.00s
Test 6 ... 77.50% 	 Time: 28.38s
Test 7 ... 71.50% 	 Time: 28.96s
Test 8 ... 74.50% 	 Time: 28.21s
Test 9 ... 78.00% 	 Time: 28.18s
Test 10 ... 70.50% 	 Time: 27.02s
Total: 75.20%
Average time: 28.48s
